#  Reading Excel Data Via the Knowledge Network

Python provides plenty of tools for reading and visualising different kinds of tabular data. Here we demonstrate using Requests, Pandas, xlrd, and matplotlib to load, manipulate, and plot Excel data from the [CSIRO Knowledge Network](https://kn.csiro.au/).

This notebook was developed by the [OzNome Project](https://research.csiro.au/oznome/) and is available on [GitHub](https://github.com/oznome/jupyter-examples). The filtering widget was produced in conjunction with the OzNome project as part of the [CSIRO EUDM](https://www.csiro.au/en/Research/EF/Areas/Electricity-grids-and-systems/Economic-modelling/Energy-Use-Data-Model) project.

## Imports and Configuration

In [ ]:
import pandas, xlrd, requests, json
from pandas import np
import matplotlib.pyplot as plt
pandas.set_option('display.max_columns', 500)
%matplotlib inline

## Getting the Data

First, request a metadata record in JSON format from the Knowledge Network.

Please consult the [Knowledge Network documentation](https://confluence.csiro.au/display/OFW/Knowledge+network) for API usage, as this is outside the scope of this notebook.

In [ ]:
response = requests.get("http://kn.csiro.au/api/dataset?id=http%3A%2F%2Foznome.csiro.au%2Fid%2Fdata%2Fdata-gov-au%2Fsaimos-biological-and-flow-cytometry-data-collected-from-ctd-stations-in-south-australia-i-20142")
json_data = response.json()

Exploring the JSON metadata will let us find some data of interest:

In [ ]:
json_data

The requests library represents JSON data as nested Python dictionaries. This is great for programmatic data access, but terrible for printing. Lets create a class (from https://stackoverflow.com/questions/18873066/pretty-json-formatting-in-ipython-notebook) to make our JSON easier to explore:

In [ ]:
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict) or isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

This will let us view any JSON data as a tree with collapsible nodes.

In [ ]:
RenderJSON(json_data)

<div class="alert alert-info">
What does this data set represent?
</div>

**Hint:** Look for the `notes` field.

<div class="alert alert-info">
Try to find and expand the `resources` record. What resources are there for this data object?
</div>

**Hint:** You can either find the resources in the full display, or take advantage of the recursive nature of JSON and print only the resources section (shown below).

In [ ]:
RenderJSON(json_data['resources'])

Let's retrieve the resource data for picophytoplankton. By using a Python list comprehension we can query a particular Excel workbook that is referenced in the metadata record:

In [ ]:
url = [resource for resource in json_data["resources"] if "Picophytoplankton" in resource["name"]][0]["url"]
url

Then using requests we can query the specific Excel workbook and open it using xlrd:

In [ ]:
r = requests.get(url)
book = xlrd.open_workbook(file_contents=r.content)

Then list the various sheets contained in this workbook:

In [ ]:
book.sheet_names()

## Exploring and Cleaning the Data

Using Pandas we can read one of these sheets and do some exploratory data analysis, starting with listing the available columns:

In [ ]:
dataframe = pandas.read_excel(url, sheetname='Converted_CLEAN')
dataframe.columns

The dataframe describe function tells us more information about these columns:

In [ ]:
dataframe.describe(include='all')

Sometime it is useful to build widgets to help explore data and make a notebook user friendly. You can define widgets inline in a notebook but it is also easy to define widgets in a separate Python file and then import them:

In [ ]:
from FilteringWidget import FilteringWidget

The above code creates a widget class called FilteringWidget which we can then use to create a widget (have a look at the file in the Jupyter Python viewer if you are curious):

In [ ]:
filtered = FilteringWidget(dataframe, ['Station', 'Depth (m)'])

Interacting with this widget produces a filtered dataframe available by accessing the dataframe property. For example we can restrict the dataframe to the NRSKAI station

In [ ]:
filtered.dataframe

Notice that some values are displayed as '-', and that this coincides with the '*missing sample*' comment field? Let's replace those '-' values with NaN:

In [ ]:
filtered_frame = filtered.dataframe.replace('-', np.nan)

Thinking about what this data might be we can make some experimental assumptions. Perhaps Rep is experiment repeat number.

We can further restrict the data frame to particular organisms by specifying specific columns:

In [ ]:
filtered_frame = filtered_frame[[ 'Synechococcus ','Prochlorococus', 'Picoeukaryotes', 'Rep', 'Depth (category)']]

In [ ]:
filtered_frame

It looks like there is data for all the organisms in "Rep 2" so lets filter to that:

In [ ]:
filtered_frame = filtered_frame.loc[filtered_frame['Rep'] == 2]

In [ ]:
filtered_frame

We won't need the rep column:

In [ ]:
filtered_frame.pop('Rep');

The current index looks like a unique identifier for the experiment but we also want to index by depth (it makes the plots simpler). So lets give the current index a name and add a second level index by depth:

In [ ]:
# Give the index a name
filtered_frame.index.name = 'experiment'

In [ ]:
# Create a standard column from the current index
filtered_frame.reset_index(level=0, inplace=True)

In [ ]:
# Create the multi-index from named columns
filtered_frame.set_index(['experiment', 'Depth (category)'], inplace=True)

## Plotting the Data
Finally we can produce some plots. Here we create a stacked bar chart showing the counts of picophytoplankton types at different depths for each of the experiments:

In [ ]:
filtered_frame.plot(kind='bar', stacked=True);

It's a little hard to compare within a single experiment in that combined plot, so lets zoom in on the SAM8SG records:

In [ ]:
filtered_frame[['SAM8' in x[0] for x in filtered_frame.index]].plot(kind='bar', title='SAM8SG');

And now lets compare surface concentrations across experiments. Note that each entry in a Pandas MultiIndex is a tuple. Depth is the second index, so we compare against `x[1]` in the list comprehension part of the query:

In [ ]:
filtered_frame[['Surface' in x[1] for x in filtered_frame.index]].plot(kind='bar', title='Surface');